In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal

In [4]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

Now let's create project and resource to keep track of our work

In [10]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [6]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [7]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [8]:
res_local.make_dir()

In [11]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [12]:
peregrine = streamm.Resource('peregrine')

In [13]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [14]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [15]:
p3ht_et.set_resource(peregrine)

In [16]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [17]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [22]:
bbTh.particles[0].rsite = 'term_'
bbTh.particles[1].rsite = 'func_'
bbTh.particles[3].rsite = 'term_'
bbTh.particles[5].rsite = 'termcap_'
bbTh.particles[6].rsite = 'funccap_'
bbTh.particles[8].rsite = 'termcap_'

Set some properties of the molecule to keep track of the parts

In [19]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)

                particle_i.paramkey = 'CA'
                particle_i.resname = "SCP2"
                particle_i.residue = 1
                c_cnt +=1 
            if( particle_i.symbol == 'S' ):
                particle_i.fftype = 'S'
                particle_i.resname = "ThS"
                particle_i.residue = 2
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)

                particle_i.fftype = 'HA'
                particle_i.resname = "HA"
                particle_i.residue = 3
                h_cnt +=1 
                

Build hexane

In [20]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [21]:
bbHex.particles[0].rsite = 'rg_'
bbHex.particles[1].rsite = 'rgcap_'

In [23]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.paramkey = 'CT'
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.paramkey = 'HC'
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Looks good let's set up some calculations

In [ ]:
calc_i = streamm.Gaussian('g_methane_HF')

In [ ]:
Methane_example.add_calc(calc_i) # Add it to the project 

In [ ]:
calc_i.strucC = ME               # set the strucC to the structure container 

In [ ]:
print calc_i.tag

Let's use the remote resource to run this calculation 

In [ ]:
calc_i.set_resource(peregrine)

* home : directory is the root directory for the calculation/project
* template : directory of template input and run files to be modified to run the calculation 
* materials : directory to store structure files (.xyz) 
* launch : directory to temporarly store files before they are copied to the remote resouce 
* scratch : directory to run the calculation
* storage : directory to store completed calculation data

In [ ]:
pprint(calc_i.dir)

In [ ]:
calc_i.make_dir()

In [ ]:
print calc_i.dir['launch']

In [ ]:
os.chdir(calc_i.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian_peregrine2.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'launch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
calc_i.load_str('templates','com')        
calc_i.load_str('templates','run')

In [ ]:
calc_i.properties['commands'] = 'HF/3-21G SP'
calc_i.properties['charge'] = 0
calc_i.properties['spin_mult'] = 1
calc_i.properties['coord'] = calc_i.strucC.write_coord()

In [ ]:
calc_i.replacewrite_prop('com','input','com','%s.com'%(calc_i.tag))

In [ ]:
calc_i.properties['input_com'] = calc_i.files['input']['com']
calc_i.replacewrite_prop('run','scripts','run','%s.pbs'%(calc_i.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(calc_i.tag)
calc_i.add_file(file_type,file_key,file_name)

In [ ]:
file_type = 'output'
file_key = 'fchk'
file_name = "%s.fchk"%(calc_i.tag)
calc_i.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(calc_i.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(calc_i.dir['launch'])

In [ ]:
calc_i.push()

In [ ]:
calc_i.run()

Cool. While that is in the queue let's setup some more jobs

Let's also run a NWChem calculation

In [ ]:
nwchem_i = streamm.NWChem('nw_methane_HF')

In [ ]:
Methane_example.add_calc(nwchem_i)

In [ ]:
nwchem_i.strucC = ME

In [ ]:
print nwchem_i.tag

In [ ]:
nwchem_i.set_resource(peregrine)

In [ ]:
pprint(nwchem_i.properties['scratch'])

In [ ]:
nwchem_i.make_dir()

In [ ]:
print nwchem_i.dir['launch']

In [ ]:
os.chdir(nwchem_i.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

In [ ]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_opt.properties['task'] = 'SCF '
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [ ]:
pprint(nwchem_i.properties)

In [ ]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

In [ ]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(nwchem_i.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(nwchem_i.dir['launch'])

In [ ]:
nwchem_i.push()

In [ ]:
nwchem_i.run()

Okay we have a couple 

In [ ]:
Methane_example.check()

In [ ]:
nwchem_i.analysis()

In [ ]:
print nwchem_i.properties['alpha_energies']

In [ ]:
print nwchem_i.properties['N_alpha_occ']

In [ ]:
Methane_example.store()

In [ ]:
Methane_example.pull()

In [ ]:
os.chdir(nwchem_i.dir['home'])
Methane_example.dump_json()

Neat!

Now let's optimize the structure and calculate the ESP charges

In [ ]:
nwchem_opt = streamm.NWChem('nw_methane_OPT')

In [ ]:
Methane_example.add_calc(nwchem_opt)

In [ ]:
nwchem_opt.strucC = ME

In [ ]:
print nwchem_opt.tag

In [ ]:
nwchem_opt.set_resource(peregrine)

In [ ]:
pprint(nwchem_opt.properties['scratch'])

In [ ]:
nwchem_opt.make_dir()

In [ ]:
print nwchem_opt.dir['launch']

In [ ]:
os.chdir(nwchem_opt.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_opt.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_opt.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
nwchem_opt.load_str('templates','nw')        
nwchem_opt.load_str('templates','run')

In [ ]:
nwchem_opt.properties['basis'] = '6-31g'
nwchem_opt.properties['method'] = 'UHF'
nwchem_opt.properties['charge'] = 0
nwchem_opt.properties['spin_mult'] = 1
nwchem_opt.properties['task'] = 'SCF optimize'
nwchem_opt.properties['coord'] = nwchem_opt.strucC.write_coord()

In [ ]:
pprint(nwchem_opt.properties)

In [ ]:
nwchem_opt.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_opt.tag))

In [ ]:
nwchem_opt.properties['input_nw'] = nwchem_opt.files['input']['nw']
nwchem_opt.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_opt.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_opt.tag)
nwchem_opt.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(nwchem_opt.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(nwchem_opt.dir['launch'])

In [ ]:
nwchem_opt.push()

In [ ]:
nwchem_opt.run()

In [ ]:
nwchem_opt.check()

In [ ]:
print nwchem_opt.meta

In [ ]:
Methane_example.check()

In [ ]:
Methane_example.store()

In [ ]:
Methane_example.pull()

In [ ]:
nwchem_opt.analysis()

In [ ]:
print nwchem_opt.strucC.write_xyz_str()

In [ ]:
os.chdir(nwchem_opt.dir['materials'])

In [ ]:
nwchem_opt.strucC.tag = '{}_{}'.format(nwchem_opt.strucC.tag,nwchem_opt.tag)

In [ ]:
nwchem_opt.strucC.write_xyz()

In [ ]:
nwchem_esp = streamm.NWChem('nw_methane_ESP')

In [ ]:
ME_OPT = streamm.Buildingblock('methane_nw_methane_OPT')

In [ ]:
ME_OPT.read_xyz()

In [ ]:
print(ME.write_xyz_str())

In [ ]:
Methane_example.add_calc(nwchem_esp)

In [ ]:
nwchem_esp.strucC = ME_OPT

In [ ]:
print nwchem_esp.tag

In [ ]:
nwchem_esp.set_resource(peregrine)

In [ ]:
pprint(nwchem_esp.properties['scratch'])

In [ ]:
nwchem_esp.make_dir()

In [ ]:
print nwchem_esp.dir['launch']

In [ ]:
os.chdir(nwchem_esp.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
nwchem_esp.load_str('templates','nw')        
nwchem_esp.load_str('templates','run')

In [ ]:
nwchem_esp.properties['basis'] = '6-31g'
nwchem_esp.properties['method'] = 'UHF'
nwchem_esp.properties['charge'] = 0
nwchem_esp.properties['spin_mult'] = 1
nwchem_esp.properties['task'] = 'esp'
nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

In [ ]:
pprint(nwchem_esp.properties)

In [ ]:
nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

In [ ]:
nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_esp.tag)
nwchem_esp.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(nwchem_esp.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(nwchem_esp.dir['launch'])

In [ ]:
nwchem_esp.push()

In [ ]:
nwchem_esp.run()

In [ ]:
print nwchem_esp.tag,nwchem_esp.files['output']

In [ ]:
nwchem_esp.check()

In [ ]:
print nwchem_esp.meta

In [ ]:
Methane_example.check()

In [ ]:
Methane_example.store()

In [ ]:
Methane_example.pull()

In [ ]:
nwchem_opt.analysis()

Now we have an optimized molecular geometery and ESP charges